# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import ollama
from IPython.display import Markdown, display, update_display

import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
OLLAMA_HEADERS = {"Content-Type": "application/json"}

In [3]:
# set up environment
!ollama pull llama3.2

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [9]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

def system_prompt():
   return """
    You are a Python code tutor. You answer questions about the Python language and provide explanations. Give your responses in Markdown format."
    """
def user_prompt():
    return question

def messages():
    return [
     {"role": "system", "content": system_prompt()},
     {"role": "user", "content":user_prompt()}
    ]
def olama_payload():
    return {
        "model": MODEL_LLAMA,
        "messages": messages(),
        "stream": True
    }

def display_markdown(md):
    display(Markdown(md))

def display_stream(stream):
    for chunk in stream:
        if chunk.choices[0].delta.content:
            display_markdown(chunk.choices[0].delta.content)
            update_display(chunk.choices[0].delta.content, display_id=True)
        if chunk.choices[0].delta.role:
            display_markdown(chunk.choices[0].delta.role)
            update_display(chunk.choices[0].delta.role, display_id=True)
        if chunk.choices[0].finish_reason:
            display_markdown(chunk.choices[0].finish_reason)


In [5]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": system_prompt()},
        {"role": "user", "content": user_prompt()}
      ],
    stream=True
)
display_stream(stream)

Certainly! Let's break down the code you provided:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation

1. **Context (`yield from`)**:
   - The `yield from` statement is used in a generator function to yield all values from an iterable. It effectively delegates part of its operations to another iterator.

2. **Set Comprehension**:
   - The part inside the curly braces `{}` is a set comprehension. It constructs a set containing certain elements generated by an expression. In this case, it iterates through the `books` collection.

3. **Iteration**:
   - The expression `for book in books` iterates over each `book` object in the `books` iterable.

4. **Getting the Author**:
   - The `book.get("author")` method is called on each `book`. This method retrieves the value associated with the key `"author"`. If the key does not exist, it returns `None`.

5. **Conditional Filtering**:
   - The `if book.get("author")` part filters the results. It ensures that only those `book` objects that have a non-None value for the `"author"` key are included in the set. Essentially, it excludes any books that do not have an author specified.

6. **Set Creation**:
   - The result of the set comprehension is a set of unique author names (or `None` values filtered out). This means that even if multiple books are attributed to the same author, the author will only appear once in the resulting set.

### Purpose

The entire line of code essentially collects and yields all unique authors from a list of book dictionaries (`books`). It ensures that only those authors who are present (i.e., the `"author"` key is not `None`) are included. By using `yield from`, it's designed to be part of a generator function that can yield each author's name one at a time.

### Example

Here's a quick example to illustrate:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]

# Generator function that utilizes the code
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Usage
for author in get_unique_authors(books):
    print(author)


### Output:

Author A
Author B


In this example, the unique authors "Author A" and "Author B" are yielded from the generator function, while the book with `None` as an author is ignored.

In [15]:
# Get Llama 3.2 to answer

# response = requests.post(OLLAMA_API, json=olama_payload, headers=OLLAMA_HEADERS)
# print(response.json()['message']['content'])

print(messages())
response = ollama.chat(model=MODEL_LLAMA, messages=messages())
display_markdown(response['message']['content'])

[{'role': 'system', 'content': '\n    You are a Python code tutor. You answer questions about the Python language and provide explanations. Give your responses in Markdown format."\n    '}, {'role': 'user', 'content': '\nPlease explain what this code does and why:\nyield from {book.get("author") for book in books if book.get("author")}\n'}]


**Generator Expression Explanation**
=====================================

The given code uses a generator expression to extract author names from a list of books. Here's a breakdown:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

*   `books` is assumed to be a list or iterable containing dictionaries, each representing a book.
*   The dictionary comprehension `{...}` iterates over the `books` collection and filters out any book without an "author" key using the `if` condition.

**What does `yield from` do?**
------------------------------

`yield from` is a Python 3.3+ feature that allows you to delegate iteration to another iterable, such as a generator expression or a list comprehension.

In this case, the outer `{...}` creates a dictionary with author names as values. The inner generator expression `(...)` yields each author name from the filtered books.

The `yield from` keyword then "unfolds" these yielded values into the outer iteration, effectively yielding all the unique authors in the order they appear in the filtered list of books.

**Example Use Case**
--------------------

Suppose you have a list of book dictionaries:
```python
books = [
    {"title": "Book A", "author": "Author X"},
    {"title": "Book B", "author": None},
    {"title": "Book C", "author": "Author Y"}
]
```
 Running the generator expression with `yield from` would yield a dictionary like this:
```python
{
    'Author X': None,
    'Author Y': None
}
```

As you can see, only books with authors are included in the result.

**Equivalent Code**
-------------------

For comparison, here's an equivalent iterative approach using a list to store author names:

```python
authors = []
for book in books:
    if book.get("author"):
        authors.append(book["author"])
return dict.fromkeys(authors)
```

While this code achieves the same result, it's more verbose and less efficient than the original generator expression.